<a href="https://colab.research.google.com/github/gamesMum/Leukemia-Diagnostics/blob/master/Leukemia_Diagnosis_Alexnet_SGD_momentum_every_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Leukemia Diagnosis**

**Classification of Acute Leukemia using Pretrained Deep Convolutional Neural Networks**
Based on the implementation in the paper:

[**Human-level recognition of blast cells in acute myeloid
leukemia with convolutional neural networks**](https://www.biorxiv.org/content/10.1101/564039v1.full.pdf)

 **The Dataset used in this implementation:**


- The dataset is for AML instead of ALL
-The number of subtypes are 16
including the Normal class  

- link to the dataset https://www.kaggle.com/rashasalim/blood-smear-images-for-aml-diagnosis




# **Materials and Methods**

- peripheral blood smears were selected from 100 patients diagnosed with different subtypes
of AML at the Laboratory of Leukemia Diagnostics at Munich University Hospital between 2014 and 2017, and smears from 100 patients found to exhibit no morphological
features of hematological malignancies in the same time frame.

- The resulting digitised data consisted of multiresolution pyramidal images of a size of approximately 1 GB per scanned area of interest.
A trained examiner experienced in routine cytomorphological diagnostics at Munich University Hospital differentiated physiological and pathological leukocyte types contained
in the microscopic scans into the classification scheme (see fig 2B),
which is derived from standard morphological categories and was refined to take into account subcategories relevant for the morphological classification of AML, such as bilobed Promyelocytes, which are typical of the FAB subtype M3v.
-  Annotation was carried out on a
single-cell basis, and approximately 100 cells were differentiated in each smear
- Subimage patches of size 400 x 400 pixels (corresponding to approximately 29µm x 29µm)
around the annotated cells were extracted without further cropping or filtering, including
background components such as erythrocytes, platelets or cell fragments.
- When examining the screened blood smears, the cytologist followed the routine clinical procedure.
Overall, 18,365 single-cell images were annotated and cut out of the scan regions.

- Annotations of single-cell images provide the ground truth for training and evaluation
of our network.

- Morphological classes containing fewer than 10 images were merged with
neighbouring classes of the taxonomy.

- A subset of 1,905 single-cell images from all morphological categories were presented to a second, independent examiner, and annotated
for a second time in order to estimate inter-rater variability

**For Implementation:**
-	The network was adopted  to input image dimensions of 400 x 400 x 3
-	No further cropping or ﬁltering.
-	Retained the cardinality hyper-parameter at C = 32.
-	The ﬁnal dense layer adapted to our 10-category classiﬁcation scheme.
-	Annotations of single-cell images provide the ground truth for training and evaluation of the network
-	There are 10 classes for training and evaluation.
-	For our image classiﬁcation task, we used a Alexnet
-	The network was trained for at least 20 epochs, which took a computing time of approximately 4 days on a Nvidia GeForce GTX TITAN X GPU.
-	The test group contains 20%, and the validation group 20% of the images
-	Random rotational transformations of 0−359 degrees, as well as random horizontal and vertical ﬂips to the single-cell images in the dataset.
-	In the end the data set was augmented in such a way that each class contained approximately 10,000 images for training. (This is from the origional paper that had 15 classes, but this number increased in this implementation since some of the classes were combined)







In [0]:
#inporting the necessary libraries
import torch
from torch import nn
from torchvision import transforms, datasets, models

In [4]:
#check if CUDA is available
train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
    print("CUDA is available. Training on GPU!")
else:
    print("CUDA is not available. Training on CPU.")

CUDA is not available. Training on CPU.


In [0]:
#time to prepare the data
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision as tv

batch_size = 32
test_size = 0.2
valid_size = 0.2

mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]


#define the transforms
train_transform  = transforms.Compose([transforms.Resize((400,400)),
                                       transforms.RandomRotation(359),
                                       transforms.RandomHorizontalFlip(0.2),
                                       transforms.RandomVerticalFlip(0.2),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=mean, std=std)])

test_transforms = transforms.Compose([transforms.Resize((400, 400)),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=mean, std=std)])

train_data = datasets.ImageFolder("/kaggle/input/single-cell-morphological-dataset-of-leukocytes/blood_smear_images_for_aml_diagnosis_MOD/AML-Cytomorphology_LMU_MOD",
                                  transform = train_transform)

#obtain training indicies that will be used as testing and validation

num_train = len(train_data)
indicies = list(range(num_train))
np.random.shuffle(indicies)
split = int(np.floor(valid_size+test_size * num_train))
train_idx, test_idx, valid_idx = indicies[split:], indicies[:int(split/2)], indicies[int(split/2):split]
#define samplers for obtainig the trainig, testing and validation set
train_sampler = SubsetRandomSampler(train_idx)
test_sampler = SubsetRandomSampler(test_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                           sampler = train_sampler)
test_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                          sampler = test_sampler)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                          sampler = valid_sampler)


#  PROMYELOCYTE (PMB Promyelocyte (bilobled))
# PMO Promyelocyte), MYELOCYTE (MYB Myelocyte, MYO Myeloblast)ARE FOUND ON LEUKEMIA PATIENTS 
classes = ['BAS', 'EBO', 'EOS', 'KSC','LYT','MON', 'MYO', 'NGB', 'NGS', 'PMO']




In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt


def imshow(img):
  img = img /2+0.5 #unormalize the images
  plt.imshow(np.transpose(img, (1, 2, 0))) #convert it back from tensor to image

#get one batch of training images
dataiter = iter(train_loader) #now contains the first batch
images, labels = dataiter.next() #images=the first batch of images, labels= the first batch of labels
images = images.numpy() #convert the images to display them

#plot the imahes in the batch along with the corresponding labels
fig = plt.figure(figsize=(25,6))

for idx in np.arange(5):
  ax = fig.add_subplot(1, 5, idx+1, xticks=[], yticks=[]) #(rows, cols, index, .., ..)
  imshow(images[idx])
  ax.set_title(classes[labels[idx]])

In [0]:
#Load AlexNet pretrained model
model = models.alexnet(pretrained=True)
model

In [0]:
#freeze the model calssifier
for param in model.parameters():
  param.requires_grad = False

from collections import OrderedDict

classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(9216, 4096)),
                          ('relu', nn.ReLU()),
                          ('dropout', nn.Dropout(0.5)),
                          ('fc2', nn.Linear(4096, 4096)),
                          ('relu', nn.ReLU()),
                          ('dropout', nn.Dropout(0.5)),
                          ('fc3', nn.Linear(4096, 1024)),
                          ('relu', nn.ReLU()),
                          ('dropout', nn.Dropout(0.5)),
                          ('fc4', nn.Linear(1024, 10)),
                          ('output', nn.LogSoftmax(dim=1) )                                       
                                      ]))

model.classifier = classifier

In [0]:
from torch import optim
#Loss function and optmixation function
criterion = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
if train_on_gpu:
    model.cuda()

model

In [0]:
# number of epochs to train the model
import numpy as np
n_epochs = 200

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for images, labels in train_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            images, labels = images.cuda(), labels.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(images)
        # calculate the batch loss (comapre the values of the output model to the actual labels)
        loss = criterion(output, labels)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*images.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for images, labels in valid_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            images, labels = images.cuda(), labels.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(images)
        # calculate the batch loss
        loss = criterion(output, labels)
        # update average validation loss 
        valid_loss += loss.item()*images.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
    
    if epoch % 10 == 0:    
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, train_loss, valid_loss))
    
        # save model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min, 
            valid_loss))
            torch.save(model.state_dict(), 'model_AML_classifier.pt')
            valid_loss_min = valid_loss

In [0]:
model.load_state_dict(torch.load('model_AML_classifier.pt'))


In [0]:
#initialize the test loss
test_loss = 0.0

class_correct = list(0. for i in range(10))
class_total = list (0. for i in range(10))

#set the model to test and validation mode (no gradient descent needed)
model.eval()

for data, target in test_loader:
  #move the tensor to GPU ig available
  if train_on_gpu:
    data, target = data.cuda(), target.cuda()
  #forward pass: compute prediction output by passing the first batch of test data
  output = model(data)
  #calculate the batch size
  loss = criterion(output, target)
  #update the test loss
  test_loss += loss.item()*data.size(0)
  #convert output probabilities to output class
  _, pred = torch.max(output, 1)
  #compare the prediction to true label
  correct_tensor = pred.eq(target.data.view_as(pred))
  #conveert to numpy array and remove the extra dimention and get only the result
  correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())

  #calculate test accuracy for each object class
  for i in range(batch_size):
    try:
      label = target.data[i] #get the corresponding label from the object
      class_correct[label] += correct[i].item()
      class_total[label] += 1
    except IndexError:
      break
  
# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
  if class_total[i] > 0:
     print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
  else:
       print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))
   



In [0]:
#Visualizing a sample tested of data
dataiter = iter(test_loader)
images, labels = dataiter.next()
images.numpy()

#Move model inputs to cuda
if train_on_gpu:
    images = images.cuda()

#get sample outputs
output = model(images)
#convert probabilties to prediction class
_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy()) if not train_on_gpu else np.squeeze(preds_tensor.cpu().numpy())

# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(10):
    ax = fig.add_subplot(2, 10/2, idx+1, xticks=[], yticks=[])
    imshow(images.cpu()[idx])
    ax.set_title("{} ({})".format(classes[preds[idx]], classes[labels[idx]]),
                 color=("green" if preds[idx]==labels[idx].item() else "red"))

In [0]:
# Visualizing the result
# draw testing vs validation error